In [200]:
import spike.spike_analysis.spike_collection as sc
import spike.spike_analysis.spike_recording as sr
import spike.spike_analysis.firing_rate_calculations as fr
import spike.spike_analysis.normalization as norm
import spike.spike_analysis.single_cell as single_cell
import spike.spike_analysis.spike_collection as collection
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import behavior.boris_extraction as boris
import matplotlib.pyplot as plt
import pickle
import re

In [201]:
pd.set_option('display.max_colwidth', 0)  # 0 means unlimited in newer pandas versions

# Show all rows
pd.set_option("display.max_rows", None)

# Show all columns
pd.set_option("display.max_columns", None)

# Don’t truncate column contents
pd.set_option("display.max_colwidth", None)

# Expand the display to the full width of the screen
pd.set_option("display.width", 0)


In [202]:
event_dict_path = r'C:\Users\thoma\Code\ResearchCode\diff_fam_social_memory_ephys\spike_collection.json\event_dict.pkl'
phy_recordings_path = r'C:\Users\thoma\Code\ResearchCode\diff_fam_social_memory_ephys\spike_collection.json\recordings'

spike_collection_json_path = r'C:\Users\thoma\Code\ResearchCode\diff_fam_social_memory_ephys\spike_collection.json\spike_collection.json'

### Making arrays numpy arrays

In [203]:
import importlib
import spike.spike_analysis.spike_collection as sc

importlib.reload(sc)

<module 'spike.spike_analysis.spike_collection' from 'c:\\Users\\thoma\\Code\\ResearchCode\\diff_fam_social_memory_ephys\\spike\\spike_analysis\\spike_collection.py'>

In [204]:
# make spike collection object from event_dict and phy_recordings_path and subject_dict
sp = sc.SpikeCollection(
    path=phy_recordings_path,
    event_dict=event_dict,
    subject_dict=subject_dict,
)
sp.analyze(timebin=100)

Event arrays are not 2 dimensional numpy arrays of shape (n x 2).
Please fix.
All set to analyze


In [205]:
new_sp = sc.SpikeCollection.load_collection(spike_collection_json_path)

In [206]:
new_sp.recordings[3].event_dict

{'alone_rewarded': array([[1856335.2 , 1866335.2 ],
        [1916335.15, 1926335.15],
        [1971335.1 , 1981335.05],
        [2036335.05, 2046335.  ],
        [2091334.85, 2101334.95],
        [2141334.9 , 2151334.9 ],
        [2191334.85, 2201334.8 ],
        [2296334.75, 2306334.75],
        [2416334.75, 2426334.6 ],
        [2536334.5 , 2546334.45],
        [2646334.5 , 2656334.45],
        [2731334.25, 2741334.1 ],
        [2851334.15, 2861334.1 ],
        [2911334.05, 2921334.05],
        [2976334.  , 2986333.95],
        [3036333.9 , 3046333.9 ],
        [3111333.85, 3121333.85],
        [3211333.85, 3221333.75],
        [3306333.65, 3316333.6 ],
        [3356333.6 , 3366333.6 ],
        [1856335.2 , 1866335.2 ],
        [1916335.15, 1926335.15],
        [1971335.1 , 1981335.05],
        [2036335.05, 2046335.  ],
        [2091334.85, 2101334.95],
        [2141334.9 , 2151334.9 ],
        [2191334.85, 2201334.8 ],
        [2296334.75, 2306334.75],
        [2416334.75, 2426334.6

In [207]:
new_sp.count_neurons_per_event()

{'20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec': {'alone_rewarded': 22,
  'alone_rewarded_baseline': 22,
  'high_comp': 22,
  'high_comp_lose': 18,
  'high_comp_lose_baseline': 22,
  'high_comp_win': 22,
  'high_comp_win_baseline': 22,
  'lose': 22,
  'low_comp': 22,
  'low_comp_lose': 22,
  'low_comp_lose_baseline': 22,
  'low_comp_win': 22,
  'low_comp_win_baseline': 22,
  'overall_pretone': 22,
  'win': 22},
 '20230612_101430_standard_comp_to_training_D1_subj_1-4_t4b2L_box1_merged.rec': {'alone_rewarded': 10,
  'alone_rewarded_baseline': 10,
  'high_comp': 10,
  'high_comp_lose': 10,
  'high_comp_lose_baseline': 10,
  'high_comp_win': 9,
  'high_comp_win_baseline': 10,
  'lose': 10,
  'low_comp': 9,
  'low_comp_lose': 9,
  'low_comp_lose_baseline': 10,
  'low_comp_win': 9,
  'low_comp_win_baseline': 10,
  'overall_pretone': 10,
  'win': 9},
 '20230612_112630_standard_comp_to_training_D1_subj_1-1_t1b3L_box2_merged.rec': {'alone_rewarded': 15,
  'alone_r

In [208]:
# Count event types in long format from a SpikeCollection
def count_event_types_long_from_collection(spike_collection):
    """
    Create a long-format DataFrame from a SpikeCollection with per-recording event_dicts.

    Args:
        spike_collection (SpikeCollection): An initialized collection object.

    Returns:
        pd.DataFrame: DataFrame with columns ['Recording ID', 'Condition', '# Trials/Events']
    """
    # Build a dict in the same format your old function expects
    event_dict_combined = {}

    
    for rec in spike_collection.recordings:
        if hasattr(rec, "event_dict") and rec.event_dict:
            event_dict_combined[rec.name] = {
                event_name: len(events)
                for event_name, events in rec.event_dict.items()
            }

    # Convert to wide DataFrame
    wide_df = pd.DataFrame.from_dict(event_dict_combined, orient='index').fillna(0).astype(int)
    wide_df.index.name = "Recording ID"

    # Melt to long format
    long_df = wide_df.reset_index().melt(
        id_vars="Recording ID",
        var_name="Condition",
        value_name="# Trials/Events"
    )

    return long_df.sort_values(["Recording ID", "Condition"])

# Usage
event_counts_long_df_from_collection = count_event_types_long_from_collection(new_sp)
event_counts_long_df_from_collection.head(20)

,Recording ID,Condition,# Trials/Events
0,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,alone_rewarded,40
39,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,alone_rewarded_baseline,40
78,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,high_comp,9
117,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,high_comp_lose,1
156,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,high_comp_lose_baseline,40
585,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,high_comp_tie,0
195,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,high_comp_win,8
234,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,high_comp_win_baseline,40
273,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,lose,7
312,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,low_comp,11


In [209]:
def convert_neuron_dict_to_long_df(neuron_dict):
    """
    Convert a nested neuron count dictionary to a long-format DataFrame.
    """
    data = []
    for recording, event_counts in neuron_dict.items():
        for condition, neuron_count in event_counts.items():
            data.append({
                "Recording ID": recording,
                "Condition": condition,
                "# Neurons": neuron_count
            })
    return pd.DataFrame(data)
# Usage
neuron_counts_long_df = convert_neuron_dict_to_long_df(new_sp.count_neurons_per_event())
neuron_counts_long_df.head(20)

,Recording ID,Condition,# Neurons
0,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,alone_rewarded,22
1,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,alone_rewarded_baseline,22
2,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,high_comp,22
3,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,high_comp_lose,18
4,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,high_comp_lose_baseline,22
5,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,high_comp_win,22
6,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,high_comp_win_baseline,22
7,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,lose,22
8,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,low_comp,22
9,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,low_comp_lose,22


In [214]:
# Merge on Recording ID and Condition
df_merged = event_counts_long_df_from_collection.merge(neuron_counts_long_df, on=["Recording ID", "Condition"], how="left")
df_merged.head(20)


,Recording ID,Condition,# Trials/Events,# Neurons
0,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,alone_rewarded,40,22.0
1,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,alone_rewarded_baseline,40,22.0
2,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,high_comp,9,22.0
3,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,high_comp_lose,1,18.0
4,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,high_comp_lose_baseline,40,22.0
5,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,high_comp_tie,0,NaN
6,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,high_comp_win,8,22.0
7,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,high_comp_win_baseline,40,22.0
8,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,lose,7,22.0
9,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,low_comp,11,22.0


In [215]:
def merge_event_and_neuron_counts(event_counts_df, neuron_counts_df, min_trials=5, min_neurons=10):
    """
    Merge the long-form event and neuron count DataFrames and flag rows suitable for Wilcoxon test.
    """
    # Merge on ['Recording ID', 'Condition']
    merged = pd.merge(event_counts_df, neuron_counts_df, on=["Recording ID", "Condition"], how="outer")

    # Fill NaNs (e.g., if a neuron/event count is missing)
    merged["# Trials/Events"] = merged["# Trials/Events"].fillna(0).astype(int)
    merged["# Neurons"] = merged["# Neurons"].fillna(0).astype(int)

    # Add a flag column
    merged["Wilcoxonable"] = (merged["# Trials/Events"] >= min_trials) & (merged["# Neurons"] >= min_neurons)

    return merged

# Generate both long-format tables
neuron_counts_long_df = convert_neuron_dict_to_long_df(new_sp.count_neurons_per_event())
event_counts_long_df = count_event_types_long_from_collection(new_sp)

# Merge and tag
merged_df = merge_event_and_neuron_counts(event_counts_long_df, neuron_counts_long_df)

# Preview only those good for Wilcoxon
wilcoxon_ready = merged_df[merged_df["Wilcoxonable"]]
display(wilcoxon_ready.head(20))


,Recording ID,Condition,# Trials/Events,# Neurons,Wilcoxonable
0,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,alone_rewarded,40,22,True
1,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,alone_rewarded_baseline,40,22,True
2,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,high_comp,9,22,True
4,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,high_comp_lose_baseline,40,22,True
6,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,high_comp_win,8,22,True
7,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,high_comp_win_baseline,40,22,True
8,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,lose,7,22,True
9,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,low_comp,11,22,True
10,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,low_comp_lose,6,22,True
11,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,low_comp_lose_baseline,40,22,True
